In [35]:
import pandas as pd
import numpy as np
import timeit
import seaborn as sns
import matplotlib.pyplot as plt

Завантанження та клінінг даних

In [36]:
# === pandas ===
start_time = timeit.default_timer()
df_pandas = pd.read_csv("data.txt", sep=";", na_values="?", low_memory=False)
df_pandas.dropna(inplace=True)
df_pandas["Global_active_power"] = df_pandas["Global_active_power"].astype(float)
pandas_time = timeit.default_timer() - start_time
print(f"=== pandas === Дані завантажено та очищено за {pandas_time:.3f} с")

# === numpy ===
start_time = timeit.default_timer()
types = [
    ("Date", "U10"), ("Time", "U8"),
    ("Global_active_power", "f8"), ("Global_reactive_power", "f8"),
    ("Voltage", "f8"), ("Global_intensity", "f8"),
    ("Sub_metering_1", "f8"), ("Sub_metering_2", "f8"),
    ("Sub_metering_3", "f8")
]

data_numpy = np.genfromtxt(
    "data.txt", delimiter=";", dtype=types, names=True,
    missing_values="?", filling_values=np.nan, encoding="utf-8"
)

data_numpy = data_numpy[~np.isnan(data_numpy["Global_active_power"])]
numpy_time = timeit.default_timer() - start_time
print(f"=== numpy === Дані завантажено та очищено за {numpy_time:.3f} с")


=== pandas === Дані завантажено та очищено за 2.196 с
=== numpy === Дані завантажено та очищено за 10.346 с


Завдання №1. 
Записи, у яких Global_active_power > 5 кВт.

In [37]:
# === pandas ===
start_time = timeit.default_timer()
power_pandas = df_pandas[df_pandas["Global_active_power"] > 5]
power_pandas_time = timeit.default_timer() - start_time
print(f"=== pandas === Записів із потужністю > 5 кВт: {len(power_pandas)} | Час: {power_pandas_time:.3f} с")

# === numpy ===
start_time = timeit.default_timer()
power_numpy = data_numpy[data_numpy["Global_active_power"] > 5]
power_numpy_time = timeit.default_timer() - start_time
print(f"=== numpy === Записів із потужністю > 5 кВт: {len(power_numpy)} | Час: {power_numpy_time:.3f} с")

=== pandas === Записів із потужністю > 5 кВт: 17547 | Час: 0.006 с
=== numpy === Записів із потужністю > 5 кВт: 17547 | Час: 0.019 с


Завдання №2. Записи, де Voltage > 235 В.

In [38]:
# === pandas ===
start_time = timeit.default_timer()
voltage_pandas = df_pandas[df_pandas["Voltage"] > 235]
voltage_pandas_time = timeit.default_timer() - start_time
print(f"=== pandas === Записів із напругою > 235 В: {len(voltage_pandas)} | Час: {voltage_pandas_time:.3f} с")

# === numpy ===
start_time = timeit.default_timer()
voltage_numpy = data_numpy[data_numpy["Voltage"] > 235]
voltage_numpy_time = timeit.default_timer() - start_time
print(f"=== numpy === Записів із напругою > 235 В: {len(voltage_numpy)} | Час: {voltage_numpy_time:.3f} с")

=== pandas === Записів із напругою > 235 В: 1952491 | Час: 0.115 с
=== numpy === Записів із напругою > 235 В: 1952491 | Час: 0.189 с


Завдання №3. Записи, у яких сила струму лежить в межах 19-20 А, та, у яких пральна машина та холодильних
споживають більше, ніж бойлер та кондиціонер.

In [39]:
# === pandas ===
start_time = timeit.default_timer()
taks3_pandas = df_pandas[(df_pandas["Global_intensity"] >= 19) & (df_pandas["Global_intensity"] <= 20)]
taks3_pandas = taks3_pandas[taks3_pandas["Sub_metering_2"] > taks3_pandas["Sub_metering_3"]]
taks3_pandas_time = timeit.default_timer() - start_time
print(f"=== pandas === Струм 19-20 А і sub_metering_2 > sub_metering_3: {len(taks3_pandas)} | Час: {taks3_pandas_time:.3f} с")

# === numpy ===
start_time = timeit.default_timer()
amp_mask = (data_numpy["Global_intensity"] >= 19) & (data_numpy["Global_intensity"] <= 20)
comparison_mask = data_numpy["Sub_metering_2"] > data_numpy["Sub_metering_3"]
task3_numpy = data_numpy[amp_mask & comparison_mask]
task3_numpy_time = timeit.default_timer() - start_time
print(f"=== numpy === Струм 19-20 А і sub_metering_2 > sub_metering_3: {len(task3_numpy)} | Час: {task3_numpy_time:.3f} с")

=== pandas === Струм 19-20 А і sub_metering_2 > sub_metering_3: 2509 | Час: 0.010 с
=== numpy === Струм 19-20 А і sub_metering_2 > sub_metering_3: 2509 | Час: 0.049 с


Завдання №4. Середні величини всіх 3-х груп споживання електричної енергії з випадково вибраних 500 000 записів  



In [ ]:
# === pandas ===
start_time = timeit.default_timer()
sample = df_pandas.sample(n=500000, replace=False, random_state=42)
task4_pandas = sample[["Sub_metering_1", "Sub_metering_2", "Sub_metering_3"]].mean()
task4_pandas_time = timeit.default_timer() - start_time
print(f"=== pandas === Середні значення для 500000 випадкових записів:")
print(task4_pandas)
print(f"Час: {task4_pandas_time:.3f} с\n")

# === numpy ===
start_time = timeit.default_timer()
random_indices = np.random.choice(len(data_numpy), size=500000, replace=False)
sample = data_numpy[random_indices]
task4_numpy = {
    "Sub_metering_1": np.mean(sample["Sub_metering_1"]),
    "Sub_metering_2": np.mean(sample["Sub_metering_2"]),
    "Sub_metering_3": np.mean(sample["Sub_metering_3"])
}
task4_numpy_time = timeit.default_timer() - start_time
print(f"=== numpy === Середні значення для 500000 випадкових записів:")
print(task4_numpy)
print(f"Час: {task4_numpy_time:.3f} с")


=== pandas === Середні значення для 500000 випадкових записів:
Sub_metering_1    1.119258
Sub_metering_2    1.308912
Sub_metering_3    6.452950
dtype: float64
Час: 0.275 с

=== numpy === Середні значення для 500000 випадкових записів:
{'Sub_metering_1': np.float64(1.1186), 'Sub_metering_2': np.float64(1.30302), 'Sub_metering_3': np.float64(6.451284)}
Час: 0.180 с


Завдання №5. Записи, які:
- після 18-00 споживають більше 6 кВт за хвилину в середньому
- у яких основне споживання електроенергії у вказаний проміжок часу припадає на пральну машину, сушарку, холодильник та освітлення
- обраний кожен третій результат із першої половини та кожен четвертий результат із другої половини.

In [41]:
# === pandas ===
start_time = timeit.default_timer()
df_pandas["Time"] = pd.to_datetime(df_pandas["Time"], format="%H:%M:%S").dt.time
evening = df_pandas[df_pandas["Time"] >= pd.to_datetime("18:00:00").time()]
high_power = evening[evening["Global_active_power"] > 6]
groupp = high_power[
    (high_power["Sub_metering_2"] > high_power["Sub_metering_1"]) &
    (high_power["Sub_metering_2"] > high_power["Sub_metering_3"])
]
mid = len(groupp) // 2
first_half = groupp.iloc[:mid]
second_half = groupp.iloc[mid:]
pandas_3rd = first_half.iloc[::3]
pandas_4rd = second_half.iloc[::4]
task5_pandas = pd.concat([pandas_3rd, pandas_4rd])
pandas_task5_time = timeit.default_timer() - start_time
print(f"=== pandas === Результатів: {len(task5_pandas)} | Час: {pandas_task5_time:.3f} с")

# === numpy ===
start_time = timeit.default_timer()

def to_hour(tstr):
    return int(tstr.split(":")[0])

time_hours = np.array([to_hour(t) for t in data_numpy["Time"]])
evening_mask = time_hours >= 18
power_mask = data_numpy["Global_active_power"] > 6
combo_mask = evening_mask & power_mask

filter = data_numpy[combo_mask]
groupn = (
    (filter["Sub_metering_2"] > filter["Sub_metering_1"]) &
    (filter["Sub_metering_2"] > filter["Sub_metering_3"])
)
filtern = filter[groupn]

mid_idx = len(filtern) // 2
numpy_3rd = filtern[:mid_idx][::3]
numpy_4rd = filtern[mid_idx:][::4]
task5_numpy = np.concatenate([numpy_3rd, numpy_4rd])

task5_numpy_time = timeit.default_timer() - start_time
print(f"=== numpy === Результатів: {len(task5_numpy)} | Час: {task5_numpy_time:.3f} с")


=== pandas === Результатів: 310 | Час: 6.056 с
=== numpy === Результатів: 310 | Час: 1.450 с


### Результати експерименту

| №  | Опис завдання                                                                                                     | pandas Час (с) | numpy Час (с)  | Оцінка pandas | Оцінка numpy | Коментар                                                                 |
|----|-------------------------------------------------------------------------------------------------------------------|------------------|------------------|------------------|------------------|--------------------------------------------------------------------------|
| 1  | Завантаження та очищення даних                                                                                    | 2.122            | 11.836           | 5                | 2                | pandas значно швидше і простіше для очищення                           |
| 2  | Відбір Global_active_power > 5 кВт                                                                              | 0.009            | 0.026            | 5                | 4                | pandas зручніше, numpy трохи повільніше                              |
| 3  | Відбір Voltage > 235 В                                                                                          | 0.179            | 0.208            | 4                | 4                | обидві структури показали подібну ефективність                           |
| 4  | Струм 19–20 А, sub_metering_2 > sub_metering_3                                                                  | 0.013            | 0.056            | 5                | 3                | pandas простіше для комбінованих умов                                  |
| 5  | Середні значення 3-х sub_metering для 500 000 випадкових записів                                                | 0.268            | 0.214            | 4                | 5                | numpy швидший при числовій агрегації                                   |
| 6  | Фільтрація за часом, потужністю, приладами, з вибіркою кожного 3-го/4-го запису                                   | 6.442            | 1.482            | 2                | 4                | numpy значно швидший у складних умовах       |
